In [31]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [32]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [33]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [34]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [35]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [36]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:06,  1.10it/s]


In [37]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,8.683659,4.683659,0.000000,10.683659,-3.316341,0.683659,0.0,8.683659,...,10.683659,-3.316341,0.683659,0.0,8.683659,4.683659,0.000000,10.683659,-3.316341,0.683659
1,2025-01-22,0.0,8.683659,4.683659,0.000000,10.683659,-3.316341,0.683659,0.0,8.683659,...,10.683659,-3.316341,0.683659,0.0,8.683659,4.683659,0.000000,10.683659,-3.316341,0.683659
2,2025-01-23,0.0,3.214377,4.683659,-13.480852,3.874608,-3.316341,0.683659,0.0,-4.310841,...,-5.494008,-3.316341,0.683659,0.0,10.739595,4.683659,-13.480852,13.243223,-3.316341,0.683659
3,2025-01-24,0.0,3.214377,4.683659,-13.480852,3.874608,-3.316341,0.683659,0.0,-4.310841,...,-5.494008,-3.316341,0.683659,0.0,10.739595,4.683659,-13.480852,13.243223,-3.316341,0.683659
4,2025-01-25,0.0,3.214377,4.683659,-13.480852,3.874608,-3.316341,0.683659,0.0,-4.310841,...,-5.494008,-3.316341,0.683659,0.0,10.739595,4.683659,-13.480852,13.243223,-3.316341,0.683659


In [38]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [39]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [40]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls['partisan'] = pres_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + pres_polls['partisan'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://poll.qu.edu/poll-release?rele...",2026-01-29,2026-02-02,1191 RV,8.975187e-02,37.0,56.0,-19.0,-15.593589
1,"<a href=""https://www.pewresearch.org/politics/...",2026-01-20,2026-01-26,8512 A,7.998551e-02,37.0,61.0,-24.0,-18.230236
2,"<a href=""https://insideradvantage.com/insidera...",2026-01-31,2026-02-01,1000 LV,6.948143e-02,50.0,49.0,1.0,-15.257411
3,"<a href=""https://law.marquette.edu/poll/2026/0...",2026-01-21,2026-01-28,1003 A,5.815941e-02,42.0,58.0,-16.0,-15.066807
4,"<a href=""https://harvardharrispoll.com/wp-cont...",2026-01-28,2026-01-29,2000 RV,5.556837e-02,45.0,51.0,-6.0,-13.469203
...,...,...,...,...,...,...,...,...,...
559,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,1.588743e-20,52.0,43.0,9.0,4.635117
560,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,1.379911e-20,47.0,41.0,6.0,9.572519
561,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,1.227727e-20,49.0,36.0,13.0,8.073103
562,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,1.197995e-20,51.9,45.3,6.6,-0.542676


In [41]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [42]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://law.marquette.edu/assets/comm...",2026-01-21,2026-01-28,Trade/tariffs,1003 A,2.248271e-02,37.0,62.0,-25.0,-20.593083
1,"<a href=""https://harvardharrispoll.com/wp-cont...",2026-01-28,2026-01-29,Trade/tariffs,2000 RV,1.697169e-02,39.0,54.0,-15.0,-22.667856
2,"<a href=""https://static.foxnews.com/foxnews.co...",2026-01-23,2026-01-26,Trade/tariffs,1005 RV,1.415635e-02,37.0,63.0,-26.0,-24.345391
3,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-23,2026-01-25,Trade/tariffs,2201 RV,8.549624e-03,43.0,48.0,-5.0,-18.941801
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-22,Trade/tariffs,2226 A,6.710835e-03,34.0,57.0,-23.0,-24.631754
...,...,...,...,...,...,...,...,...,...,...
1107,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,1.476812e-14,47.0,37.0,10.0,8.368246
1108,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,8.547565e-15,48.0,37.0,11.0,9.368246
1109,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,4.955066e-15,45.0,41.0,4.0,2.368246
1110,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,2.903608e-15,48.0,35.0,13.0,11.368246


In [43]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [44]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['partisan'] = gb_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + gb_polls['partisan'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://www.cygn.al/polls/national-po...",2026-01-27,2026-01-28,1004 LV,9.573866e-02,48.0,44.0,-4.0,-5.562649
1,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-30,2026-02-02,1504 RV,8.869421e-02,44.0,40.0,-4.0,-3.930547
2,"<a href=""https://law.marquette.edu/poll/2026/0...",2026-01-21,2026-01-28,577 LV,8.446873e-02,52.0,45.0,-7.0,-6.291180
3,"<a href=""https://echelonin.wpenginepowered.com...",2026-01-22,2026-01-26,1029 LV,7.308077e-02,49.0,44.0,-5.0,-7.256105
4,"<a href=""https://static.foxnews.com/foxnews.co...",2026-01-23,2026-01-26,1005 RV,7.195624e-02,52.0,46.0,-6.0,-3.957594
...,...,...,...,...,...,...,...,...,...
287,"<a href=""https://www.cygn.al/voters-back-kingm...",2025-01-09,2025-01-12,1500 LV,2.174520e-20,44.6,46.8,2.2,0.995196
288,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,1.571682e-20,40.0,48.0,8.0,2.631627
289,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,8.045608e-21,45.0,48.0,3.0,1.048976
290,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,6.302886e-22,44.6,48.4,3.8,2.595196


In [45]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)